# Libraries

Speed up sklearn as per https://www.kaggle.com/c/tabular-playground-series-jan-2022/discussion/298241

In [ ]:
#!pip install scikit-learn-intelex

In [ ]:
#from sklearnex import patch_sklearn
#patch_sklearn()

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns

import lightgbm as lgb 
#import xgboost as xgb
from catboost import CatBoostRegressor

from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

import dateutil.easter as easter
import optuna
import math

In [ ]:
LINEAR_DATE_AUG = False

OPTUNA = False
NUM_TRIALS = 200
SEED = 5

#Holidays
HOLIDAYS = False     
NEXT_HOLIDAY = False  

SEASONS = False
WEATHER = False 

LAG_FEATURES = True

POST_PROCESSING = False
MODEL_TYPE = "Catboost"

VAL_SPLIT = "2017-12-31" #"2018-05-31"


In [ ]:
EPOCHS = 10000     
EARLY_STOPPING = 30

###########Come back to this   ---Doesnt work in CATBOOST (need to apply it)
SCALER_NAME = "MinMax"  #None MinMax  
SCALER = MinMaxScaler() 

# Load Data

In [ ]:
train_df = pd.read_csv("../input/tabular-playground-series-jan-2022/train.csv",index_col = 0)
test_df = pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv",index_col = 0)
sub = pd.read_csv("../input/tabular-playground-series-jan-2022/sample_submission.csv",index_col = 0)

test_index = test_df.index
train_index = train_df.index

gdp_df = pd.read_csv('../input/gdp-20152019-finland-norway-and-sweden/GDP_data_2015_to_2019_Finland_Norway_Sweden.csv')
gdp_df.set_index('year', inplace=True)

if HOLIDAYS:
    holidays = pd.read_csv("../input/holidays-finland-norway-sweden-20152019/Holidays_Finland_Norway_Sweden_2015-2019.csv",usecols = ["Date","Country","Name"]                      )
    holidays.rename(columns = {"Date":"date","Country":"country","Name":"holiday"},inplace= True)
    holidays["holiday"]= 1
    holidays["holiday"]= holidays["holiday"].astype("int32")
    holidays["date"] = pd.to_datetime(holidays["date"])

In [ ]:
df_weather = pd.read_csv('../input/finland-norway-and-sweden-weather-data-20152019/nordics_weather.csv', parse_dates=['date'])

In [ ]:
#Make date
train_df["date"] = pd.to_datetime(train_df["date"])
test_df["date"] = pd.to_datetime(test_df["date"])

In [ ]:
train_df.head()

# Functions 

In [ ]:
if SEASONS:
    print("Adding Seasons ")
    seasons = [1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 1]

    month_to_season = dict(zip(range(1,13), seasons))

    train_df["season"] = train_df["date"].dt.month.map(month_to_season)
    test_df["season"] = test_df["date"].dt.month.map(month_to_season)

In [ ]:
if WEATHER:
    w_feats = ['country', 'date', 'tavg','precipitation']
    #w_feats = ['country', 'date', 'precipitation', 'snow_depth', 'tavg', 'tmax','tmin']
    print("Adding weather")
    train_df = train_df.merge(df_weather[w_feats], on=['date', 'country'], how='left')
    train_df.index = train_index 
    test_df = test_df.merge(df_weather[w_feats], on=['date', 'country'], how='left')
    test_df.index = test_index 

All values are in April and Dec which aligns to Christmas and Easter 
Also mostly Kaggle Ramma and sales of hats

In [ ]:
def public_hols(df):
    df = pd.merge(df, holidays, how='left', on=['date', 'country'])
    df.fillna(value = 0,inplace=True)
    
    return df

if HOLIDAYS:
    train_df = public_hols(train_df)
    test_df = public_hols(test_df)

In [ ]:
def get_gdp(row):
    country = 'GDP_' + row.country
    return gdp_df.loc[row.date.year, country]

In [ ]:
'''def engineer(df):
    #get GDP from file 
    gdp_exponent = 1.2121103201489674 # see https://www.kaggle.com/ambrosm/tpsjan22-03-linear-model for an explanation
    def get_gdp(row):
        """Return the GDP based on row.country and row.date.year"""
        country = 'GDP_' + row.country
        return gdp_df.loc[row.date.year, country] ** gdp_exponent
    
    df["day"] = df["date"].dt.day
    df['week'] = df.date.dt.week 
    df["month"] = df["date"].dt.month
    df["year"] = df["date"].dt.year
    
    #play around with if Tree model 
    #df['dayofyear'] = df['date'].dt.dayofyear                ### This can cause noise 
    df['inverse_dayofyear'] = 365 - df['date'].dt.dayofyear    
    #df.loc[df["year"] == 2016 , "inverse_dayofyear"] = df.loc[df["year"] == 2016 , "inverse_dayofyear"]+1     #Leap year in 2016   
    #df['daysinmonth'] = df['date'].dt.days_in_month          ## Bad for all except Lightgbm

    df['quarter'] = df.date.dt.quarter
    df['weekday'] = df.date.dt.weekday
    df['day_of_week'] = df.date.dt.dayofweek  
    df['day_of_year'] = df.date.dt.dayofyear  
    df['week_of_year'] = df.date.dt.weekofyear
    df['day_of_month'] = df.date.dt.days_in_month  
    df['is_weekend'] = np.where((df['weekday'] == 5) | (df['weekday'] == 6), 1, 0)
    df['is_friday'] = np.where((df['weekday'] == 4), 1, 0)
    
    if LINEAR_DATE_AUG:
        for country in ['Finland', 'Norway']:
            df[country] = df.country == country
        df['KaggleRama'] = df.store == 'KaggleRama'
        for product in ['Kaggle Mug', 'Kaggle Sticker','Kaggle Hat']:#Kaggle Hat
            df[product] = df['product'] == product
        df.drop(["country","store","product"], axis =1, inplace = True)

        # Seasonal variations (Fourier series)
        dayofyear = df.date.dt.dayofyear
        for k in range(1, 3):
            new_df[f'sin{k}'] = np.sin(dayofyear / 365 * 2 * math.pi * k)
            new_df[f'cos{k}'] = np.cos(dayofyear / 365 * 2 * math.pi * k)
            new_df[f'mug_sin{k}'] = new_df[f'sin{k}'] * new_df['Kaggle Mug']
            new_df[f'mug_cos{k}'] = new_df[f'cos{k}'] * new_df['Kaggle Mug']
            new_df[f'hat_sin{k}'] = new_df[f'sin{k}'] * new_df['Kaggle Hat']
            new_df[f'hat_cos{k}'] = new_df[f'cos{k}'] * new_df['Kaggle Hat']

    return df'''

In [ ]:
'''def engineer_more(df):

    new_df = engineer(df)

    # End of year
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"dec{d}":
                                      (df.date.dt.month == 12) & (df.date.dt.day == d)
                                      for d in range(24, 32)})],
                       axis=1)
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"jan{d}":
                                      (df.date.dt.month == 1) & (df.date.dt.day == d) 
                                      for d in range(1, 13)})],
                       axis=1)
    
    # May
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"may{d}":
                                      (df.date.dt.month == 5) & (df.date.dt.day == d) 
                                      for d in list(range(1, 10)) + list(range(17, 25))})],
                       axis=1)
    
    # June
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"june{d}":
                                      (df.date.dt.month == 6) & (df.date.dt.day == d) 
                                      for d in list(range(6, 14))})],
                       axis=1)
    
    # Last Wednesday of June
    wed_june_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-06-24')),
                                         2016: pd.Timestamp(('2016-06-29')),
                                         2017: pd.Timestamp(('2017-06-28')),
                                         2018: pd.Timestamp(('2018-06-27')),
                                         2019: pd.Timestamp(('2019-06-26'))})
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"wed_june{d}": 
                                      (df.date - wed_june_date == np.timedelta64(d, "D"))
                                      for d in list(range(-5, 6))})],
                       axis=1)
    
    # First Sunday of November
    sun_nov_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-11-1')),
                                         2016: pd.Timestamp(('2016-11-6')),
                                         2017: pd.Timestamp(('2017-11-5')),
                                         2018: pd.Timestamp(('2018-11-4')),
                                         2019: pd.Timestamp(('2019-11-3'))})
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"sun_nov{d}": 
                                      (df.date - sun_nov_date == np.timedelta64(d, "D"))
                                      for d in list(range(0, 10))})],
                       axis=1)
    
    # Easter
    easter_date = df.date.apply(lambda date: pd.Timestamp(easter.easter(date.year)))
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"easter{d}": 
                                      (df.date - easter_date == np.timedelta64(d, "D"))
                                      for d in list(range(-2, 11)) + list(range(40, 48)) + list(range(50, 59))})],
                       axis=1)
        
    return new_df

train = engineer_more(train_df)
train['num_sold'] = train.num_sold.astype(np.float32)
test = engineer_more(test_df)
test.index = test_index


features = test.columns
print(list(features))'''

In [ ]:
# Feature engineering

gdp_df = pd.read_csv('../input/gdp-20152019-finland-norway-and-sweden/GDP_data_2015_to_2019_Finland_Norway_Sweden.csv',
                    index_col='year')
gdp_exponent = 1.2121103201489674 # see https://www.kaggle.com/ambrosm/tpsjan22-03-linear-model for an explanation
def get_gdp(row):
    """Return the GDP based on row.country and row.date.year"""
    country = 'GDP_' + row.country
    return gdp_df.loc[row.date.year, country] ** gdp_exponent

le_dict = {feature: LabelEncoder().fit(train_df[feature]) for feature in ['country', 'product', 'store']}

def engineer(df):
    """Return a new dataframe with the engineered features"""
    
    new_df = pd.DataFrame({'gdp': df.apply(get_gdp, axis=1),
                           'dayofyear': df.date.dt.dayofyear,
                           'wd4': df.date.dt.weekday == 4, # Friday
                           'wd56': df.date.dt.weekday >= 5, # Saturday and Sunday
                          })

    new_df.loc[(df.date.dt.year != 2016) & (df.date.dt.month >=3), 'dayofyear'] += 1 # fix for leap years
    
    for feature in ['country', 'product', 'store']:
        new_df[feature] = le_dict[feature].transform(df[feature])
        
    # Easter
    easter_date = df.date.apply(lambda date: pd.Timestamp(easter.easter(date.year)))
    new_df['days_from_easter'] = (df.date - easter_date).dt.days.clip(-3, 59)
    new_df.loc[new_df['days_from_easter'].isin(range(12, 39)), 'days_from_easter'] = 12 # reduce overfitting
    #new_df.loc[new_df['days_from_easter'] == 59, 'days_from_easter'] = -3
    
    # Last Wednesday of June
    wed_june_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-06-24')),
                                         2016: pd.Timestamp(('2016-06-29')),
                                         2017: pd.Timestamp(('2017-06-28')),
                                         2018: pd.Timestamp(('2018-06-27')),
                                         2019: pd.Timestamp(('2019-06-26'))})
    new_df['days_from_wed_jun'] = (df.date - wed_june_date).dt.days.clip(-5, 5)
    
    # First Sunday of November (second Sunday is Father's Day)
    sun_nov_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-11-1')),
                                         2016: pd.Timestamp(('2016-11-6')),
                                         2017: pd.Timestamp(('2017-11-5')),
                                         2018: pd.Timestamp(('2018-11-4')),
                                         2019: pd.Timestamp(('2019-11-3'))})
    new_df['days_from_sun_nov'] = (df.date - sun_nov_date).dt.days.clip(-1, 9)
    
    return new_df

train = engineer(train_df)
train['date'] = train_df.date # used in GroupKFold
train['num_sold'] = train_df.num_sold.astype(np.float32)
train['target'] = train['num_sold'] / train['gdp']
test = engineer(test_df)
test['date'] = test_df.date 

In [ ]:
#NB catboost can encode within model
categorical_feats = [
    "country","store","product",
                     #"quarter", 
                     #"weekend_ind"
                    ]

In [ ]:
def next_Holiday(country, date):

    countryHolidays = holidays[holidays["country"] ==country]["date"]
    try:
        nextDate = min([day for day in countryHolidays if day >= date])
    except ValueError:
        #no next holiday
        nextDate = date
    return (nextDate - date).days

if NEXT_HOLIDAY:
    print("Applying Next Holiday")
    train['daysTillHoliday'] = train.apply(lambda x: next_Holiday(x['country'], x['date']), axis=1)
    test['daysTillHoliday'] = test.apply(lambda x: next_Holiday(x['country'], x['date']), axis=1)

In [ ]:
def SMAPE(y_true, y_pred):
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)

In [ ]:
def scale_data(X_train, X_test= None, test=None,):
     
    scaler= SCALER
    
    #this can be X or X_train 
    X_train_s = scaler.fit_transform(X_train)

    if X_test is None: #full train 
        test_s = scaler.transform(test)
        return X_train_s, test_s
    
    else: # validation 
        X_test_s = scaler.transform(X_test)
    
    return   X_train_s , X_test_s

In [ ]:
def create_lag(DAYS,df):
    df[f"shift{DAYS}"] = df.groupby(["store","product","country"])["target"].shift(DAYS,fill_value = 0)
    return df

In [ ]:
def rolling_mean_std(roll_window, df):
    shift_days=1
    col_name = 'rolling_'+str(shift_days)+'_'+str(roll_window)   
    df[col_name+"_mean"] = df.groupby(["store","product","country"])["target"].shift(shift_days).rolling(roll_window).mean()
    df[col_name+"_std"] = df.groupby(["store","product","country"])["target"].shift(shift_days).rolling(roll_window).std()
    df[col_name+"_median"] = df.groupby(["store","product","country"])["target"].shift(shift_days).rolling(roll_window).median()
    #df[col_name+"_max"] = df.groupby(["store","product","country"])["target"].shift(shift_days).rolling(roll_window).max()
    #df[col_name+"_min"] = df.groupby(["store","product","country"])["target"].shift(shift_days).rolling(roll_window).min()
    #df[col_name+"diff"] = df[col_name+"_max"] - df[col_name+"_min"]
    
#     df[col_name+"_mean"] = df.groupby(["store","product","country"])["target"].shift(0).rolling(7).mean()    
#     df[col_name+"_std"] = df.groupby(["store","product","country"])["target"].shift(0).rolling(7).std()
#     df[col_name+"_median"] = df.groupby(["store","product","country"])["target"].shift(0).rolling(7).median()
    return df.fillna(0,inplace = True)

In [ ]:
def day_roll(df,day_shift,roll_window):
    #Shift values and rolling mean
    for day in days_shift:
        create_lag(day,df)
    
    rolling_mean_std(roll_window,df)

    return df

# Shift Days 

In [ ]:
#days_shift = [1, 2,3,4,5,7,14,30, 60]
days_shift = [i for i in range(1,30)]
roll_window = 7

In [ ]:
if LAG_FEATURES:
    print("Running Lag features")
    train = day_roll(train,days_shift,roll_window)

### One HotEncoder and date index 

In [ ]:
# Catboost does encoding in training 
train = pd.get_dummies(train,columns= categorical_feats)
test = pd.get_dummies(test,columns= categorical_feats)

In [ ]:
#train.index = train["date"]
#train.drop("date",axis=1,inplace=True)
#test.index = test["date"]
#test.drop("date",axis=1,inplace=True)

# Set Features 

In [ ]:
features = train.columns 
features = features.drop(["num_sold","date","target","gdp"])

features_base = test.columns
features_base = features_base.drop(["date","gdp"])

print([i for i in features])

In [ ]:
train[["store","product","country"]]= train_df[["store","product","country"]]
test[["store","product","country"]]= test_df[["store","product","country"]].values

# Split and Scale

In [ ]:
X = train[features]
y= train["target"]

In [ ]:
X_train = train[train["date"]<=VAL_SPLIT][features]
X_test = train[train["date"]>VAL_SPLIT][features]
y_train= train[train["date"]<=VAL_SPLIT]["target"]
y_test= train[train["date"]>VAL_SPLIT]["target"]

In [ ]:
params = {
'loss_function': 'Poisson',
'eval_metric': 'Poisson',
'learning_rate': 0.04082853714917523,
'l2_leaf_reg': 0.11616644663434757,
'depth': 3,
'boosting_type': 'Plain',
'bootstrap_type': 'Bernoulli',
'min_data_in_leaf': 16,
'one_hot_max_size': 9,
'subsample': 0.9756940817316571,
'od_type': 'Iter',       # type of overfitting detector
'od_wait': 40,
'has_time': True         # use the order of the data (ts), do not permute
}

'''# NEw Optuna AWS 
params = {'loss_function': 'Tweedie:variance_power=1.2337418466853298',
 'eval_metric': 'Tweedie:variance_power=1.1443753953439761',
 'learning_rate': 0.06799537783216487,
 'l2_leaf_reg': 0.35748081310296065,
 'depth': 5,
 'boosting_type': 'Plain',
 'bootstrap_type': 'Bernoulli',
 'min_data_in_leaf': 8,
 'subsample': 0.31307811702026744 
}'''


# Models 

In [ ]:
'''params0 = {'objective': 'regression', # Manual optimization
           'force_row_wise': True,
           'max_bin': 400, # need more bins than days in a year
           'verbosity': -1,
           'seed': 1,
           'bagging_seed': 3,
           'feature_fraction_seed': 2,
           'learning_rate': 0.018,
           'lambda_l1': 0,
           'lambda_l2': 1e-2,
           'num_leaves': 18,
           'feature_fraction': 0.710344827586207,
           'bagging_fraction': 0.47931034482758617,
           'bagging_freq': 3,
           'min_child_samples': 20}

model = lgb.LGBMRegressor(**params0, n_estimators = 2000)
model.fit(X_train[features_base],np.log1p(y_train), eval_set= [(X_test[features_base],np.log1p(y_test))])
val_preds = np.expm1 (model.predict(X_test))
smape = SMAPE(y_test,val_preds)'''

In [ ]:
def fit_model(X_train,y_train, seed_num, params,X_test= None,y_test = None, test_df= None):
    model = CatBoostRegressor(**params,
                      iterations=EPOCHS,
                  #cat_features=categorical_feats, 
                      random_seed=seed_num
                 )

    
    if test_df is None:     
        scaler = SCALER
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        model.fit(
            X_train, np.log1p(y_train),
            verbose=0
            )

        val_preds = np.expm1 ( model.predict(X_test))
        smape = SMAPE(y_test,val_preds)
        print(" SMAPE:", smape)
        
        return val_preds,  smape, model
        
    else: 
        scaler = SCALER
        X_train = scaler.fit_transform(X_train)
        test_df = scaler.transform(test_df)

        model.fit(
            X_train, np.log1p(y_train),
            verbose=0
            )
        
        test_preds = np.expm1 ( model.predict(test_df))

        return test_preds

In [ ]:
val_preds, smape, model_train = fit_model(X_train,y_train, 0, params,X_test,y_test)
val_preds = val_preds*train[train["date"]>VAL_SPLIT]["gdp"]

## Residual A

In [ ]:
feature_importance = pd.DataFrame({'feature_importance': model_train.get_feature_importance(), 'feature_names': X_train.columns}).sort_values(by=['feature_importance'],
                                                                                                                        ascending=False)
plt.figure(figsize=(25,10))
plt.bar(x = feature_importance['feature_names'], height=feature_importance["feature_importance"])
plt.xticks(rotation = 90)
plt.show()

In [ ]:
res_base = train[train["date"]>VAL_SPLIT]["num_sold"] - val_preds 

plt.figure(figsize=(25,10))
sns.lineplot(x = train.loc[X_test.index,"date"], y = res_base,ci = None)
plt.axhline(0, c="r")
#sns.lineplot(x = train.loc[X_test.index,"date"], y = train["num_sold"],ci = None )
plt.show()

In [ ]:
plt.figure(figsize=(25,10))
sns.scatterplot(x =train[train["date"]>VAL_SPLIT]["num_sold"], y = val_preds)
plt.show()

### Note: 
* Increase variance over time - log  / box cox transformation could help 

In [ ]:
# Fit on Full Train data
test_preds = fit_model(X[features_base],y, 0, params, test_df= test[features_base])

sub_base = sub.copy(deep= True)
sub_base["num_sold"]  = test_preds * test["gdp"]
sub_base["num_sold"] = sub_base["num_sold"].round()
sub_base.to_csv("sub_base.csv")
sub_base

In [ ]:
plt.figure(figsize=(25,10))

sns.lineplot(x = test["date"], y = sub_base["num_sold"])
sns.lineplot(x = train["date"], y = train["num_sold"])
plt.title("Full base model predictions ")
plt.show()

## Random Seeding
Variation of [adamwurdits](https://www.kaggle.com/adamwurdits/tps-01-2022-catboost-w-optuna-seed-averaging) 

In [ ]:
def seed_CV(X_train,y_train, params,X_test=None, y_test=None, test_df = None):
    
    if test_df is not None:
        seed_preds = np.zeros((SEED, len(test_df)))
    else:
        smape_score = []
        seed_preds = np.zeros((SEED, len(y_test)))
    
    for i in range(SEED):
        print(f"##### SEED {i} #######")
        
        if test_df is None:
            val_preds, smape, model_train = fit_model(X_train,y_train, i, params,X_test,y_test)
            smape_score.append(smape)
            seed_preds[i] = val_preds

        else: 
            test_preds = fit_model(X_train, y_train, i,params, test_df= test_df)
            seed_preds[i] = test_preds            
    
    if test_df is None:
        print("\nFinal SMAPE:",np.mean(smape_score))
        return np.mean(smape_score), np.mean(seed_preds,axis =0)
        
    return np.mean(seed_preds,axis =0)

In [ ]:
smape , seed_train_preds  = seed_CV(X_train, y_train, params, X_test, y_test , test_df =None)

In [ ]:
# Fit on Full Train data - NB !!!! this excludes lag and shift - therefore less accurate than CV above
seed_test_preds  = seed_CV(X[features_base], y, params , test_df =test[features_base])
seed_test_preds

In [ ]:
sub_base_seed = sub.copy(deep= True)
sub_base_seed["num_sold"]  = seed_test_preds* test["gdp"]
sub_base_seed["num_sold"] = sub_base_seed["num_sold"].round()
sub_base_seed.to_csv("sub_base_seed.csv")
sub_base_seed

# Group Cross Validation 
As per [ambrosm](https://www.kaggle.com/ambrosm/tpsjan22-06-lightgbm-quickstart#Re-training,-inference-and-submission)

In [ ]:
from sklearn.model_selection import GroupKFold

In [ ]:
'''cv = GroupKFold(n_splits=4)

smape_score = []
for idx, (train_idx,val_idx) in enumerate (cv.split(train, groups=train.date.dt.year)):
    X_train , X_test = train.iloc[train_idx] ,train.iloc[val_idx]
    y_train , y_test = train.iloc[train_idx]["target"] , train.iloc[val_idx]['target']

    print(f"\n Training: {X_train.date.dt.year.unique()}, Predicting: {X_test.date.dt.year.unique()}" )

    smape , seed_train_preds  = seed_CV(X_train[features], y_train, params, X_test[features], y_test , test_df =None)
    
    smape_score.append(smape)
print("\nFinal Smape", np.mean(smape_score))'''

In [ ]:
'''plt.figure(figsize=(25,10))
sns.lineplot(x= X_test["date"], y = seed_train_preds* test["gdp"],ci = None)
sns.lineplot(x= X_test["date"], y = y_test, ci = None)'''

# Multi-Step Recursive Seed

In [ ]:
#all_df [  (all_df["date"]>= start_date -delta) & (all_df["date"]< start_date+delta) ][["store","country","product","date",'rolling_0_7_std']]

In [ ]:
FREQUENCY = 1 #days to predict --each  loop 
start_date = min(test["date"]) 
end_date = max(test["date"])

In [ ]:
def multi_step_recursive(start_date, end_date, freq, sub, train_i, test_i):
    delta = pd.DateOffset(days = 1)

    all_df = pd.concat([train_i.assign(ds="a"),test_i.assign(ds="b")],axis =0)
    
    #Shift values and rolling 
    if LAG_FEATURES:
        day_roll(all_df,days_shift,7)

        
    while start_date <= end_date:

        #Select slice to predict
        test_split = all_df [  (all_df["date"]>= start_date ) & (all_df["date"]< start_date+delta) ][features]
        
        #display(test_split)
        
        
        X = all_df[ all_df["date"]< start_date]
        y = all_df[ all_df["date"]< start_date]["target"]
        
        test_preds = fit_model(X[features],y, 0, params, test_df= test_split)
        
        #Add predicted-test data to X and predicted-target to y
        all_df.loc[test_split.index, "target"]  = test_preds
        
        sub.loc[test_split.index , "num_sold"] = test_preds
    
        #Shift values and rolling 
        if LAG_FEATURES:
            day_roll(all_df,days_shift,7)

        #update start date
        start_date += delta
        
    return sub  ,smape

In [ ]:
sub_recursive, seed_smape  = multi_step_recursive(start_date, end_date, FREQUENCY, sub.copy(deep=True), train, test)
sub_recursive

In [ ]:
sub_recursive["num_sold"] =sub_recursive["num_sold"]* test["gdp"]
sub_recursive["num_sold"] =sub_recursive["num_sold"].round()
sub_recursive.to_csv("sub_recursive.csv")
sub_recursive

In [ ]:
plt.figure(figsize=(20,10))

#sns.lineplot(data= train["num_sold"] ,label="Train Actual",ci=None)
sns.lineplot(data = sub_base,x = test["date"] , y = "num_sold", label ="Base prediction" ,ci=None)
sns.lineplot(data = sub_base_seed,x = test["date"] , y = "num_sold", label ="Seed prediction" ,ci=None)
sns.lineplot(data = sub_recursive,x = test["date"] , y = "num_sold", label ="Recursive prediction" ,ci=None)

# Multi Step by Store/Product / Country 

In [ ]:
def split_models(split_on, sub_df, train ,test):    

    split_smape=0
    
    # split training on product/ store/ country
    for split in train[split_on].unique():
        print(f"\nPredicting for {split_on} {split}")

        train_split= train[train[split_on] ==split]
        test_split =test[test[split_on] ==split]
        
        #train on Full dataset
        final_predictions , smape = multi_step_recursive(start_date, end_date, FREQUENCY, sub_df.loc[test_split.index], train_split, test_split, feature_list= features)
        split_smape += smape/train[split_on].nunique()
        
        sub_df.loc[test_split.index,"num_sold"] = final_predictions["num_sold"]

    print(f"\n Final mean smape:",split_smape)
    
    return split_smape, sub_df, model

In [ ]:
# store_smape, sub_store, model = split_models("store", sub.copy(deep=True), train ,test)
# sub_store

In [ ]:
# product_smape, sub_product, model = split_models("product", sub.copy(deep=True), train ,test)

In [ ]:
# sub_product

In [ ]:
# country_smape, sub_country, model = split_models("country", sub.copy(deep=True), train ,test)

# All Together - store.product & country Split
This is 18 Models, 1 for each variation of store, product and country

In [ ]:
import itertools
all_splits = list(itertools.product(['KaggleMart', 'KaggleRama'],['Kaggle Mug', 'Kaggle Hat', 'Kaggle Sticker'],['Finland', 'Norway', 'Sweden']))

In [ ]:
def split_models_ALL(split_on, sub_df):    

    split_smape=0
    train["preds"]= 0
    split_dict = {}

    # split training on product/ store/ country
    for split in split_on:
        print(f"\nPredicting for store: {split[0]}, product: {split[1]}, country: {split[2]} ")

        train_split= train[ (train["store"] == split[0]) & (train["product"] == split[1]) & (train["country"] == split[2])]
        test_split =test[ (test["store"] == split[0]) & (test["product"] == split[1]) & (test["country"] == split[2])]

        X_train = train_split.iloc[train_split.index<=VAL_SPLIT,:].drop("num_sold", axis=1)
        X_test = train_split.iloc[train_split.index>VAL_SPLIT,:].drop("num_sold", axis=1)
        y_train= train_split.iloc[train_split.index<=VAL_SPLIT,:]["num_sold"]
        y_test= train_split.iloc[train_split.index>VAL_SPLIT,:]["num_sold"]


        #run model for each split type
        smape, test_preds, train_preds  = seed_CV(X_train,y_train, X_test, y_test, test_split, params)
        print(f"\n split smape:",smape)

        split_smape += smape/len(all_splits)
        sub_df.loc[test_split.index,"num_sold"] = test_preds
        split_dict[split] = smape
        

    print(f"\n final all_split smape:",split_smape)
    
    return split_smape, sub_df , split_dict

In [ ]:
#sub = pd.read_csv("../input/tabular-playground-series-jan-2022/sample_submission.csv",index_col = 0)
#all_smape, all_sub, split_dict = split_models_ALL(all_splits, sub.copy(deep=True))

In [ ]:
'''print("  store ,       product ,       country:  score")
print("--------------------------")
for i in split_dict.items():
    print(i[0],":",'{:.2f}'.format(i[1]))'''

In [ ]:
# print("One Shot Smape :",seed_smape )
# print("Store SMAPE  :",store_smape )
# print("Product SMAPE :",product_smape )
# print("Country SMAPE :",country_smape )

# Training Visualization

In [ ]:
'''plt.figure(figsize=(20,10))

#sns.lineplot(data= train["num_sold"] ,label="Train Actual",ci=None)
sns.lineplot(data = sub_base,x = test["date"] , y = "num_sold", label ="Base prediction" ,ci=None)
sns.lineplot(data = sub_store,x = test["date"] , y = "num_sold", label ="Store split prediction" ,ci=None)
sns.lineplot(data = sub_recursive,x = test["date"] , y = "num_sold", label ="Recursive split prediction" ,ci=None)
sns.lineplot(data = sub_product, x= test["date"] , y = "num_sold", label ="Product split prediction" ,ci=None)
sns.lineplot(data = sub_country, x= test["date"] , y = "num_sold", label ="Country split prediction" ,ci=None)
#sns.lineplot(data = all_sub, x= test["date"] , y = "num_sold", label ="All  split prediction" ,ci=None)

plt.show()'''

In [ ]:
'''plt.figure(figsize=(25,10))

sns.lineplot(data= train["num_sold"] ,label="Train Actual",ci=None)
sns.lineplot(data =all_sub, x= test1["date"], y = "num_sold",label="Final Predictions" ,ci=None) 
plt.title("Actual and Predicted Sales")

plt.show()'''

# Submission 

In [ ]:
# sub_recursive.to_csv("sub_recursive.csv")

# sub_country.to_csv("submission_country.csv")
# sub_product.to_csv("submission_product.csv")
# sub_store.to_csv("submission_store.csv")
